# 4.3 Automatic Differentiation
Getting better = Minimizing a loss function  
With neural networks we choose loss functions that are differentiable with respect to our parameters.  
The autograd package expedites this work by automatically calculating derivatices.

In [1]:
import torch
from torch.autograd import Variable

the mapping $y = 2\mathbf{x}^{\top}\mathbf{x}$ with respect to the column vector $\mathbf{x}$

In [3]:
x = Variable(torch.arange(4, dtype=torch.float32).reshape((4, 1)), requires_grad=True)
print(x)

tensor([[0.],
        [1.],
        [2.],
        [3.]], requires_grad=True)


변수 x를 생성하고 초기값 할당  
`requires_grad = True`

In [4]:
y = 2*torch.mm(x.t(), x)
print(y)

tensor([[28.]], grad_fn=<MulBackward0>)


(x.t() = $x^{\top}$임을 상기)  
x의 shape는